<a href="https://colab.research.google.com/github/Adxtxp/aditipendurkar.github.io/blob/main/Data%20Warehouse%20Intern%20Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assessment Title: Data Warehouse Intern Assessment: ETL & Data Modeling

Case Study: Sales Data Transformation for Analytical Reporting

Name: Aditi Pendurkar

Univeristy: Singapore Univeristy of Social Sciences

Date: 13/09/2025

In [150]:
import csv
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


I used a notepad and saved the file as a csv file. Alternatively, the tables could have been created in database with create table syntax.The script was written in python.

In [151]:
#read data
dataOrders=pd.read_csv('orders.csv')
dataProducts=pd.read_csv('products.csv')




In [152]:
dataOrders.info()
print("Data Order shape:", dataOrders.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   OrderID     7 non-null      int64  
 1   ProductID   7 non-null      object 
 2   CustomerID  7 non-null      object 
 3   OrderDate   7 non-null      object 
 4   Quantity    7 non-null      int64  
 5   Price       7 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 468.0+ bytes
Data Order shape: (7, 6)


In [153]:
dataProducts.info()
print("Data Products shape:", dataProducts.shape)
#1.productID error in the column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductID    4 non-null      object
 1   ProductName  4 non-null      object
 2   Category     4 non-null      object
 3   Cost         4 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 260.0+ bytes
Data Products shape: (4, 4)


In [154]:
print(dataOrders)

   OrderID ProductID CustomerID OrderDate  Quantity  Price
0     1001      P001       C101  5/1/2024         2   15.5
1     1002      P002       C102  5/1/2024         1   25.0
2     1003      P001       C103  6/1/2024         3   15.5
3     1004      P003       C101  7/1/2024         1   50.0
4     1005      P002       C104  8/1/2024         2   25.0
5     1006      P004       C105  8/1/2024         1  120.0
6     1007      P001       C102  9/1/2024         1   15.5


In [155]:
print(dataProducts)

  ProductID ProductName     Category  Cost
0      P001    Keyboard  Peripherals    10
1      P002       Mouse  Peripherals    18
2      P003     Monitor     Displays    40
3      P004      Webcam  Peripherals    80


**Qn 1:Transform**

**"Calculate the Revenue for each order line item (Quantity * Price)."**

In [156]:
# transform
dataOrders['Revenue'] =dataOrders['Quantity']*dataOrders['Price']


print(dataOrders)


   OrderID ProductID CustomerID OrderDate  Quantity  Price  Revenue
0     1001      P001       C101  5/1/2024         2   15.5     31.0
1     1002      P002       C102  5/1/2024         1   25.0     25.0
2     1003      P001       C103  6/1/2024         3   15.5     46.5
3     1004      P003       C101  7/1/2024         1   50.0     50.0
4     1005      P002       C104  8/1/2024         2   25.0     50.0
5     1006      P004       C105  8/1/2024         1  120.0    120.0
6     1007      P001       C102  9/1/2024         1   15.5     15.5


**QN 1: Transform**


**"Derive OrderYear, OrderMonth, and OrderDay from the OrderDate."**

In [157]:
dataOrders['OrderDate'] = pd.to_datetime(dataOrders['OrderDate'])
dataOrders['OrderYear'] = dataOrders['OrderDate'].dt.year
dataOrders['OrderMonth'] = dataOrders['OrderDate'].dt.month
dataOrders['OrderDay'] = dataOrders['OrderDate'].dt.day

selectedData = dataOrders[['OrderDate', 'OrderYear', 'OrderMonth', 'OrderDay']]

print(selectedData)


   OrderDate  OrderYear  OrderMonth  OrderDay
0 2024-05-01       2024           5         1
1 2024-05-01       2024           5         1
2 2024-06-01       2024           6         1
3 2024-07-01       2024           7         1
4 2024-08-01       2024           8         1
5 2024-08-01       2024           8         1
6 2024-09-01       2024           9         1


**Join the orders and products data to enrich the sales records
with product details (e.g., ProductName, Category).**

In [158]:
mergedData = pd.merge(dataOrders, dataProducts, on='ProductID', how='left')

print(mergedData)

   OrderID ProductID CustomerID  OrderDate  Quantity  Price  Revenue  \
0     1001      P001       C101 2024-05-01         2   15.5     31.0   
1     1002      P002       C102 2024-05-01         1   25.0     25.0   
2     1003      P001       C103 2024-06-01         3   15.5     46.5   
3     1004      P003       C101 2024-07-01         1   50.0     50.0   
4     1005      P002       C104 2024-08-01         2   25.0     50.0   
5     1006      P004       C105 2024-08-01         1  120.0    120.0   
6     1007      P001       C102 2024-09-01         1   15.5     15.5   

   OrderYear  OrderMonth  OrderDay ProductName     Category  Cost  
0       2024           5         1    Keyboard  Peripherals    10  
1       2024           5         1       Mouse  Peripherals    18  
2       2024           6         1    Keyboard  Peripherals    10  
3       2024           7         1     Monitor     Displays    40  
4       2024           8         1       Mouse  Peripherals    18  
5       2024   

**Load: Load the transformed data into a structured format. You are
encouraged to use a SQLite database to store the processed data,
designing appropriate tables for a simple data warehouse schema
(e.g., a fact table for sales and dimension tables for products and
dates). If SQLite is not feasible, clearly explain your chosen storage
method (e.g., flat files in a structured format like Parquet or CSV).**

**I used SQLlite database.**

In [159]:
conn= sqlite3.connect("orderDetails.db")
cur= conn.cursor()
cur.execute("PRAGMA foreign_keys = ON;")




In [160]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS dataProduct (
  productKey   INTEGER PRIMARY KEY,
  ProductID     TEXT UNIQUE NOT NULL,
  ProductName   TEXT NOT NULL,
  Category      TEXT NOT NULL,
  Cost          REAL NOT NULL
);
CREATE TABLE IF NOT EXISTS dateDetails (
  dateKey  INTEGER PRIMARY KEY,
  dateISO  TEXT NOT NULL,
  Year      INTEGER NOT NULL,
  Month     INTEGER NOT NULL,
  Day       INTEGER NOT NULL
);
CREATE TABLE IF NOT EXISTS transactionsDetails (
  salesKey    INTEGER PRIMARY KEY,
  dateKey     INTEGER NOT NULL,
  productKey  INTEGER NOT NULL,
  quantity    INTEGER NOT NULL,
  unitPrice   REAL NOT NULL,
  FOREIGN KEY (dateKey) REFERENCES dateDetails(dateKey),
  FOREIGN KEY (productKey) REFERENCES dataProduct(productKey)
);
""")
conn.commit()

In [161]:
#cleaning data sets
dataProduct_df = (
    dataProducts[['ProductID','ProductName','Category','Cost']]
    .drop_duplicates()
    .reset_index(drop=True)
)
dataProduct_df.insert(0, "productKey", range(1, len(dataProduct_df)+1))

# Extract date parts and create dateKey and dateISO in dataOrders first
dataOrders['OrderDate'] = pd.to_datetime(dataOrders['OrderDate'])
dataOrders['dateKey'] = dataOrders['OrderDate'].dt.strftime("%Y%m%d").astype(int)
dataOrders['dateISO'] = dataOrders['OrderDate'].dt.strftime("%Y-%m-%d")
dataOrders['Year']    = dataOrders['OrderDate'].dt.year
dataOrders['Month']   = dataOrders['OrderDate'].dt.month
dataOrders['Day']     = dataOrders['OrderDate'].dt.day


dateDetails_df = (
    dataOrders[['dateKey','dateISO','Year','Month','Day']]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [162]:
prod_map = dict(zip(dataProduct_df['ProductID'], dataProduct_df['productKey']))
transactions_df = mergedData.copy()
transactions_df['dateKey']   = transactions_df['OrderDate'].dt.strftime("%Y%m%d").astype(int)
transactions_df['productKey'] = transactions_df['ProductID'].map(prod_map)
transactions_df = transactions_df.rename(columns={
    'OrderID':'salesKey',
    'Quantity':'quantity',
    'Price':'unitPrice'
})[['salesKey','dateKey','productKey','quantity','unitPrice']]

In [163]:
dataProduct_df.to_sql("dataProduct", conn, if_exists="replace", index=False)
dateDetails_df.to_sql("dateDetails", conn, if_exists="replace", index=False)
transactions_df.to_sql("transactionsDetails", conn, if_exists="replace", index=False)


7

In [164]:
conn = sqlite3.connect("orderDetails.db")

check = pd.read_sql_query("""
SELECT *
FROM dateDetails

""", conn)

print(check)

    dateKey     dateISO  Year  Month  Day
0  20240501  2024-05-01  2024      5    1
1  20240601  2024-06-01  2024      6    1
2  20240701  2024-07-01  2024      7    1
3  20240801  2024-08-01  2024      8    1
4  20240901  2024-09-01  2024      9    1


**Analytical Query (SQL):
o Write a SQL query against your designed data warehouse schema to
answer the following business question: "What is the total revenue for each
product category for each month in the data?"**

In [165]:
conn = sqlite3.connect("orderDetails.db")

check = pd.read_sql_query("""
SELECT
  d.Year,
  d.Month,
  p.Category,
  ROUND(SUM(t.quantity * t.unitPrice), 2) AS total_revenue
FROM transactionsDetails AS t
JOIN dateDetails       AS d ON t.dateKey    = d.dateKey
JOIN dataProduct       AS p ON t.productKey = p.productKey
GROUP BY d.Year, d.Month, p.Category
ORDER BY d.Year, d.Month, p.Category
;
""", conn)

print(check)

   Year  Month     Category  total_revenue
0  2024      5  Peripherals           56.0
1  2024      6  Peripherals           46.5
2  2024      7     Displays           50.0
3  2024      8  Peripherals          170.0
4  2024      9  Peripherals           15.5


In [167]:
conn.commit()
conn.close()

**Qn-Data Modeling:
o In your solution, clearly outline the schema you designed for your data
warehouse. Describe the tables (e.g., FactSales, DimProduct, DimDate), their
columns, and the relationships between them. A simple star schema is
expected.**

**Data Modelling:**

Schema:
transactionsDetails(FactSales):

    salesKey, dateKey, productKey, quantity,unitPrice

salesKey- Primary Key
dateKey- Foreign Key
productKey- Foreign Key

dataProduct (DimProduct):

    productKey, ProductID, ProductName, Category, Cost   

productKey- Primary Key
    

dateDetails (dateDetails) :

    dateKey, dateISO, Year, Month, Day
dateKey- Primary Key

There is a 1 to many relationship between transactionDetail.productKey and DataProduct.productKey.

There is also a 1 to many realtionship between transactionDetails.dateKey and dateDeatils.dateKey



 **Qn:Dashboarding Use Case:**

 **How your designed data model (FactSales, DimProduct, DimDate or
similar) is suitable for efficient dashboarding.**


1. Summarisation of Data
    - Data can be sorted into year, month and category for easier visualisation and understanding. This is especially important when dealing with large datasets.
    - can also be presented in the form of charts and graphs


2. Simple Joins using foreign key and primary key
    - Keeps queries simple and makes data processing faster and predictable. This allows for fewer columns, faster scans and computationally cheaper joins.




3. Rich Time Intelligence
   - dateDetails has Year, Month and Day so tools can build monthly/yearly trends and enable period comparisons without the need to recompute data on the fly.



**What are three key metrics or visualizations you would propose for
this sales dashboard? Explain why each is important for
understanding sales performance. (You don't need to build the
dashboard, just describe your approach).**


1. Monthly revenue by category
  - This would help with demand analysis as it helps businesses see seasonal patterns and trends. Comparison of categories and understanding of this data enable well-informed decisions on inventory allocation, as well as the needs of the customer base. This, in turn, allows for more tailored products to be released and increases their likelihood of being well-received. This would increase revenue.


2. Top Products by Revenue
  - Bar graphs can be used to display this data. This would highlight the best performing sales, which can be prioritised to stock up and drive promotions. The highest earning products within a chosen period of time can be evaluated.
  - Moreover, this knowledge can be used to spot opportunities for new promotions and sales


3. Product's Average Price(X axis) vs Revenue (Y axis)
  - A scatter plot/ linear regression can be used to model this data. This would highlight whether most of the revenue comes from lower-priced, high-volume goods or higher-priced, low-volume goods. This helps to refine the marketing strategy and product positioning decisions to maximise sales and revenues.
  - The linear regression line's metrics, such as the R2 score, indicate how strongly the price relates to the revenue. A high R2 score indicates that revenue is strongly related to price.